# 模拟



In [ ]:
library(magrittr)
library(tibble)
library(plyr)
set.seed(888)

概率论与赌博之间有不为人知的渊源。蒙特卡罗(Monte Carlo)，欧洲“赌博之都”，就是一个不幸的证明。然而，称之为澳门模拟或者香港赌马模拟也并没有让我感觉好多少。我更愿意简单称之为“模拟”。

模拟被广泛用于 (i) 检测有限样本渐近理论的预测效果； (ii) 自举法(bootstrap),一种数据驱动的自主推断方法；
(iii) 生成非标准分布； (iv) 估算没有解析表达式的积分。在本章中我们侧重讲解(i) 和 (ii)，而 (iii) 和 (iv)
会在下一章 **积分** 中提到。

从现在起，我们将开始写脚本。**脚本** 是有特定目的的程序文本。一个有上千行的脚本并不是从头到尾写出来的，而是递归地逐步完成的。我们将整个复杂的工作分散为各个易于操作的任务，写一小段片段，测试它，在必要的时候再将它编写为一个自定义函数(user-defined function)。之后，各个分散的部分再由上层结构整合起来。(这个过程就像建造一艘空客380的飞机: 它的引擎和机翼在英国制造，机身在德国制造，其他部件的制造也分散在世界各地；然后所有的部件在法国的图卢兹被整合组装。)最后，在文本中加入注释以增加可读性。如果没有注释，在一个月后你重新打开脚本的时候，你就会忘记之前执行的操作，得从头理解一遍程序。

```{prf:example}
祖冲之 (429--500 AD)，一个古代的中国数学家，计算出 $\pi$ 介于 3.1415926 和 3.1415927 之间，其计算的精确度直到900年后才被打破。他使用的是一种确定性近似演算法(deterministic approximation algorithm)。现在假如我们给祖冲之展示一台现代计算机，他能够怎样实现更好的估计效果呢？

```

In [ ]:
require(plotrix)
require(grid)

plot(c(-1, 1), c(-1, 1), type = "n", asp = 1, xlab = "x", ylab = "y")
rect(-1, -1, 1, 1)
draw.circle(0, 0, 1)
points(x = runif(100) * 2 - 1, y = runif(100) * 2 - 1, pch = 20, col = "blue")


站在大数定律的肩膀上，我们知道 $\pi$ 可以用随机算法来估计。由

$$
\frac{\pi r^{2}}{\left(2r\right)^{2}}=E\left[\boldsymbol{1}\left\{ x^{2}+y^{2}\leq1\right\} \right],
$$

可以得出 $\pi=4\times E[ 1 \{  x^{2}+y^{2}\leq1 \}]$ 。上式中的数学期望是未知的，我们在此可以通过模拟进行估计。


In [ ]:
n <- 10000000
Z <- 2 * matrix(runif(n), ncol = 2) - 1 # uniform distribution in [-1, 1]

inside <- mean(sqrt(rowSums(Z^2)) <= 1) # the center of the cirle is (0, 0)
cat("The estimated pi = ", inside * 4, "\n")


只要计算机的内存允许，样本数量可以设置得越大越好。这样可以用平均数的平均数进行不断的迭代以实现更高的精确度。


## 有限样本计算

在实际生活中，样本数量是有限的。有限样本的数据的分布取决于样本的数量 $n$ ，仅有在少数的案例中才能找到简单的数学表达式。
幸运的是，当样本数量非常大时，表达式可以被简化。渐近理论就是这样用于估计有限样本分布的工具。无论是在计量经济学或者统计学中，目前为止，它是分析估计量和检验最有用的方法。模拟是评价估计精确度的一种方法。

尽管有真实数据的实证例子也能用于展示，但人造数据更加方便有效。计量经济学中主流的范式是假设数据是基于模型生成的。之后我们作为研究者，检验估计量能与未知参数表征的模型的吻合程度。在模拟当中，我们可以完全掌握数据的生成过程(data generation process, dgp)以及真实参数。然而，在现实例子中，我们无从得知真实的模型，所以我们不能直接评估参数估计的质量和准确性。

(如果我们只侧重于预测，那就是另一种情况了。就像我们在机器学习中经常遇到的情况那样，我们就把数据分为两个部分，一个用于建模和估计，另一部分用于验证。)




```{prf：example}

在OLS经典理论中，通常 $X$ 被视为固定的回归量并仅仅关注误差项的随机性。不过当代计量经济学教科书则会强调随机的 $X$ 更适合于计量经济学的各种应用。在严密的教科书语言中， $X$ 的矩通常被直接表述为 $E[X_i X_i'] < \infty$ 。

* 当 $X$ 服从形状系数为1.5的帕累托分布[Pareto distribution](https://en.wikipedia.org/wiki/Pareto_distribution)时， OLS估计量的渐近推断理论---一致性和渐近正态性是否还成立呢？

* 形状系数介于1到2之间的帕累托分布有无限的方差和有限的总体均值，因此这个例子违背了大多数教科书中对于OLS的基本假设。

我们可以写一个脚本来研究这个问题，步骤如下：

 1. 给定样本规模，得到OLS $`b_hat`$ 和与之对应的 `t值`.
 2. 将 `t值` 封装为一个自定义函数，这样就可以反复调用该函数。s.
 3. 给定样本规模，报告两种分布下的样本量
 4. 同样将第三步封装为自定义函数，为不同的样本规模做准备
 5. 建立一个总体结构来串联起各个函数
 6. 增加注解和操作手册
 ```


In [ ]:
# the workhorse functions
simulation <- function(n, type = "Normal", df = df) {
  # a function gives the t-value under the null
  if (type == "Normal") {
    e <- rnorm(n)
  } else if (type == "T") {
    e <- rt(n, df)
  }

  X <- cbind(1, VGAM::rpareto(n, shape = 1.5))
  Y <- X %*% b0 + e
  rm(e)

  bhat <- solve(t(X) %*% X, t(X) %*% Y)
  bhat2 <- bhat[2] # parameter we want to test

  e_hat <- Y - X %*% bhat
  sigma_hat_square <- sum(e_hat^2) / (n - 2)
  sig_B <- solve(t(X) %*% X) * sigma_hat_square
  t_value_2 <- (bhat2 - b0[2]) / sqrt(sig_B[2, 2])

  return(t_value_2)
}

# report the empirical test size
report <- function(n) {
  # collect the test size from the two distributions
  # this function contains some repetitive code, but is OK for such a simple one
  TEST_SIZE <- rep(0, 3)

  # e ~ normal distribution, under which the t-dist is exact
  Res <- plyr::ldply(.data = 1:Rep, .fun = function(i) simulation(n, "Normal"))
  TEST_SIZE[1] <- mean(abs(Res) > qt(.975, n - 2))
  TEST_SIZE[2] <- mean(abs(Res) > qnorm(.975))

  # e ~ t-distribution, under which the exact distribution is complicated.
  # we rely on asymptotic normal distribution for inference instead
  Res <- plyr::ldply(.data = 1:Rep, .fun = function(i) simulation(n, "T", df))
  TEST_SIZE[3] <- mean(abs(Res) > qnorm(.975))

  return(TEST_SIZE)
}

## the super structure
# set the parameters
Rep <- 1000
b0 <- matrix(1, nrow = 2)
df <- 1 # t dist. with df = 1 is Cauchy

# run the calculation of the empirical sizes for different sample sizes
NN <- c(5, 10, 200, 5000)
RES <- plyr::ldply(.data = NN, .fun = report)
names(RES) <- c("exact", "normal.asym", "cauchy.asym") # to make the results readable
RES$n <- NN
RES <- RES[, c(4, 1:3)] # beautify the print
print(RES)


第一列是，我们在误差正态时使用精确分布理论来找到的临界值 (依据 $t$ 分布)。第二列依然基于误差项的正态分布性质，但是这次的临界值依靠正态分布得到(基于渐近估计理论)。当样本量很小时就会观测到明显的规模失真；但是随着样本量的增加，偏差随之减少。

当误差分布是柯西分布时，所谓的 **精确分布** 就不再精确了——只有当误差项正态且独立于 $x$ 时才是精确的。
如果我们尝试使用渐近正态估计，我们会发现渐近估计无法成立，因为随着样本规模的增大，检验并不会收敛于5%的临界值。

当我们在此设计模拟时，尽管误差项的分布各不相同， $X$ 总是服从帕累托分布的。当我们查看表格的时候，我们会发现 $X$ 的分布实际上并不影响结果，这是为什么呢？ 因为有

$$
\sqrt{n} (\hat{\beta} - \beta_0) |X = (X'X/n)^{-1}  (X' e /\sqrt{n}),
$$

以 $x$ 为条件的，系数向量的第 $k$ 个元素为

$$
\widehat{\beta}_{k}|X=\eta_{k}'\widehat{\beta}|X
\sim N\left(\beta_{k},\sigma^{2}\left(X'X\right)_{kk}^{-1}\right).
$$

因此 $t$ 统计量为

$$
T_{k}  =\frac{\widehat{\beta}_{k}-\beta_{k}}{\sqrt{s^{2}\left[\left(X'X\right)^{-1}\right]_{kk}}}\\
  =\frac{\widehat{\beta}_{k}-\beta_{k}}{\sqrt{\sigma^{2}\left[\left(X'X\right)^{-1}\right]_{kk}}}\cdot\frac{\sqrt{\sigma^{2}}}{\sqrt{s^{2}}}\\
  =\frac{\left(\widehat{\beta}_{k}-\beta_{k}\right)/\sqrt{\sigma^{2}\left[\left(X'X\right)^{-1}\right]_{kk}}}{\sqrt{\frac{e'}{\sigma}M_{X}\frac{e}{\sigma}/\left(n-K\right)}}.
$$

虽然 $X'X/n$ 不会收敛于一个稳定的概率极限，自正则的 $t$ 统计量依然存在。
忽视 $X$ 的分布类型，当误差项是正态分布时，上述表达式的分子服从一个标准正态分布，而分母则服从 $\chi^2$ 分布， 且两者相互独立。最后得出的统计量服从 $t$ 分布。



## 自举法

自举法(Bootstrap)最早来源于{cite}`efron1979bootstrap`,是一个在估计和推断理论中影响力巨大的思想。在此我们简单进行介绍。教科书的阐述可以参考
{cite}`cameron2005microeconometrics` (第11章)。

令 $X_1, X_2, \ldots, X_n \sim F$ 是服从分布 $F$ 的 $n$ 个独立同分布样本。统计量 $T_n(\theta)\sim G_n(\cdot, F)$ 的有限样本分布通常取决于样本的大小 $n$ 和已知的真实分布 $F$ 。渐近理论通过极限

$$
G(\cdot, F) := \lim_{n\to\infty} G_n(\cdot, F).
$$ 

来估计 $G_n(\cdot, F)$ 。
特别地，如果 $T_n(\theta)$ 是 **渐近枢轴量**(*asymptotically pivotal*)，那么 $G_n(\cdot, F)$ 独立于 $F$ 且它简化为 $G(\cdot)$ 。


In [ ]:
runif(10) %>%
  ecdf() %>%
  plot(, xlim = c(0, 1), main = "ECDF for uniform distribution")


自举法(Bootstrap)将 $G_n(\cdot, F)$ 中的未知分布 $F$ 用真实分布的相合估计量 $F_n$ 替代，而不是使用分布的极限；例如实证分布函数
$\hat{F}_n(\cdot) = n^{-1} \sum_{i=1}^n 1\{\cdot \leq X_i\}$ 。自举法推断是从自举分布(bootstrap distribution)

$$
G^{*}_n(\cdot):= G_n(\cdot, \hat{F}_n)
$$

中得出的。
自举法是一个模拟问题。在一个独立同分布样本中，我们给每一个观测值同等的权重，这称为 **非参数自举法** (*nonparametric bootstrap*)。
不过，自举法有许多不同的变体以应对不同情况的需求。
在一个相关数据集中，例如时间序列数据[{cite}`chang2004bootstrap`]、聚类数据或网络(clustering data or networks)，我们必须调整抽样方式以保存数据的相关结构。如果是在回归问题的背景下，我们固定自变量，只对残差进行自举，则被称为 **参数自举法**(*parametric bootstrap*)。此外，如果误差项是异方差，那么 $X$ 和 $\hat{e}$ 之间的关系可以用 **原始自举法** (*wild bootstrap* )[{cite}`davidson2010wild`] 保持。


In [ ]:
n <- 9 # sample size
boot_Rep <- 3 # bootstrap 3 times

real_sample <- rnorm(n) # the real sample
d0 <- tibble(no = 1:n, x = real_sample)
print(d0)

d_boot <- list() # save the bootstrap sample
for (b in 1:boot_Rep) {
  boot_index <- sample(1:n, n, replace = TRUE)
  d_boot[[b]] <- tibble(no = boot_index, x = real_sample[boot_index])
}

d_boot %>% as_tibble(, .name_repair = "minimal") %>% print()


在大多数情况下，自举法的 **相合性** 是可以证明的：我们感兴趣的统计量与它的自举法的结果会收敛于一个相同的渐近分布；或者对于所有的$a$ ， $G^*_n(a) \stackrel{p}{ \to } G(a)$ ，从而 $G(a)$ 是连续函数。不过，若是统计量的分布在极限时不连续，那么自举法的相合性就不再成立，这种情况的自举法是不稳定的。比如在弱工具变量时，简单自举法不能复原两阶段最小二乘估计量的渐近分布。

有许多复杂的方法来解决自举法的相合性问题，本讲我们暂不介绍。

### 自举法估计

用 `ply` 来处理自举法十分便捷——这个程序包中有成套的函数来进行反复的模拟。此外，R包 [boot](http://cran.r-project.org/web/packages/boot/index.html) 提供了一个一般性的函数 `boot()`供开发者使用。

当估计量的解析式难以求解或者编程时，自举法就能发挥出它自身的优势和作用。

```{prf:example}

{cite}`heckman1977sample`的两阶段法是样本选择中最流行的估计量之一。令最终等式形如

$$
y_i = x_i \beta + u_i
$$

选择等式(selection equation)为：

$$
D_i = z_i \gamma + v_i
$$

要获得一个点估计量，我们只需要运行一个Probit模型，预测参与的概率(probability of participation)，之后再用 $x_i$ 和 $\lambda (\hat{D}_i)$ 对 $y_i$ 进行OLS回归就能得到结果。(其中 $\lambda(\cdot)$ 是inverse Mill's ratio）不过，从Heckman (1979)的原文中可以看出，二阶段估计量的渐近方差非常复杂。与其写出解析式，不如对于方差使用自举法。
```

In [ ]:
# the dataset comes from
# Greene( 2003 ): example 22.8, page 786
library(sampleSelection)
data(Mroz87)
# equations
selection_eq <- lfp ~ -1 + age + faminc + exper + educ
outcome_eq <- wage ~ exper + educ

# Heckman two-step estimation
heck <- sampleSelection::heckit(selection_eq, outcome_eq, data = Mroz87)
print(lmtest::coeftest(heck))


下方是进行单次自举的函数。为了便捷高效，我们保留点估计 `heckit` ，但忽略估计的方差。(这是一个图省事的方法，如果我们手动编写Heckit点估计的函数，也能得到相同的结果) 在此之后，就是进行重复的运算。

In [ ]:
n <- nrow(Mroz87)
boot_heck <- function() {
  indices <- sample(1:n, n, replace = T) # resample the index set
  Mroz87_b <- Mroz87[indices, ] # generate the bootstrap sample
  heck_b <- sampleSelection::heckit(selection_eq, outcome_eq, data = Mroz87_b)
  return(coef(heck_b))
}
# repeat the bootstrap
boot_Rep <- 199
Heck_B <- plyr::ldply(.data = 1:boot_Rep, .fun = function(i) boot_heck())

# collect the bootstrap outcomes
Heck_b_sd <- apply(Heck_B, 2, sd)[1:7]
print(Heck_b_sd)


解析式法和自举法的标准误差有相似的性质。二者都是渐近相合的，同时自举估计量也可以直接用于计算置信区间。

### 自举检验

自举法在推断中十分重要。实际上，如果检验统计量是渐近枢轴量，那么与解析渐进估计相比，自举法有着高阶的优点[{cite}`hall1996bootstrap`]。这是可以严格证明的。此外，如果检验统计量是渐近枢轴量，自举的假设检验可能会比解析渐近的算法更加精确。

```{prf:example}

我们用自举法对于样本均值进行假设检验。在此我们使用 $t$ 统计量。样本的分布既不是 **正态分布** 也不是 **以0为中心的卡方分布** 。它将会证明自举法的检验比渐近估计更加精确。

我们首先准备函数
```

In [ ]:
# the t-statistic for a null hypothesis mu
T_stat <- function(Y, mu) sqrt(n) * (mean(Y) - mu) / sd(Y)

# the bootstrap function
boot_test <- function(Y, boot_Rep) {
  # INPUT
  # Y: the sample
  # boot_Rep: number of bootstrap replications

  n <- length(Y)
  boot_T <- rep(0, boot_Rep)

  # bootstrap in action
  for (r in 1:boot_Rep) {
    indices <- sample.int(n, n, replace = T) # resampling the index
    resampled_Y <- Y[indices] # construct a bootstrap artificial sample
    boot_T[r] <- abs(T_stat(resampled_Y, mean(Y)))
    # the bootstrapped t-statistic
    # mu is replaced by "mean(Y)" to mimic the situation under the null
  }

  # bootstrap critical value
  boot_critical_value <- quantile(boot_T, 1 - alpha)
  # bootstrap test decision
  return(abs(T_stat(Y, mu)) > boot_critical_value)
}


自举检验的一个关键点在于无论假设的参数是否为真实值，我们都必须引入原假设。
因此，自举法的 $t$ 统计量为：

$$
T^{*}_{n} = \frac{\bar{X^{*}} - \bar{X}} { s^{*} / \sqrt{n} }.
$$

也就是说，自举法 $t$ 统计量以 $F_n$ 的样本均值 $\bar{X}$ 为中心，而不是 $\theta$ ， $F$ 的总体均值。这是因为在自举法中，“真实”的分布是 $F_n$ ；如果我们错误地让 $t$ 统计量以 $\theta$ 为中心，那么当原假设为False时，检验将变得不可靠。

下面了代码展示了三种决策方式下的拒绝概率。


In [ ]:
compare <- function() {
  # this function generates a sample of n observations
  # and it returns the testing results from three decision rules

  if (distribution == "normal") {
    X <- rnorm(n)
  }
  else if (distribution == "chisq") {
    X <- rchisq(n, df = 3) - 3
  }

  t_value_X <- T_stat(X, mu) # T-statistic

  # compare it to the 97.5% of t-distribution
  exact <- abs(t_value_X) > qt(0.975, df = n - 1)
  # compare it to the 97.5% of normal distribution
  asym <- abs(t_value_X) > 1.96
  # decision from bootstrap
  boot_rule <- boot_test(X, boot_Rep)

  return(c(exact, asym, boot_rule))
}

# set the parameters
n <- 20
distribution <- "normal"
boot_Rep <- 199
MC_rep <- 2000
alpha <- 0.05
mu <- 0

# Monte Carlo simulation and report the rejection probability
res <- plyr::ldply(.data = 1:MC_rep, .fun = function(i) compare())
colnames(res) <- c("exact", "asym", "bootstrap")
print(colMeans(res))


这里将名义显著性水平定为5%。通过程序报告实际的显著性水平——拒绝假设的数量占据重复实验总次数的比例。实际显著性水平与名义水平越相近，检验的准确率就越高。

当暗含的分布为 $\chi^2$ 时，很难导出精确的分布的解析式。不过，我们仍然可以比较渐近和自举的显著性水平。

In [ ]:
distribution <- "chisq"

res <- plyr::ldply(.data = 1:MC_rep, .fun = function(i) compare())
colnames(res) <- c("exact?", "asym", "bootstrap")
print(colMeans(res))


这里的“精确检验”同样不 **精确** 。 渐近检验的工作原理更可靠，而自举法能够更接近5%的显著性水平。

## 拓展阅读

Efron and Hastie: 第 10 和 11 章

## 参考文献
